<a href="https://colab.research.google.com/github/asahilkamble108/ANN-mini/blob/main/2-differentiable-quantum-computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture

# Comment this out if you don't want to install pennylane from this notebook
!pip install pennylane

# Comment this out if you don't want to install matplotlib from this notebook
!pip install matplotlib

# Differentiable quantum computing with PennyLane

In this tutorial we will:

* learn step-by-step how quantum computations are implemented in PennyLane,
* understand parameter-dependent quantum computations ("variational circuits"),
* build our first quantum machine learning model, and
* compute its gradient.

We need the following imports:

In [31]:
import pennylane as qml
from pennylane import numpy as np

In [41]:
import pandas as pd

df = pd.read_csv('tor.csv')
print(df.head())
print(df.shape)
print(df.dtypes)

   Source IP   Source Port  Destination IP   Destination Port   Protocol  \
0  10.0.2.15         53913   216.58.208.46                 80          6   
1  10.0.2.15         53913   216.58.208.46                 80          6   
2  10.0.2.15         53913   216.58.208.46                 80          6   
3  10.0.2.15         53913   216.58.208.46                 80          6   
4  10.0.2.15         53913   216.58.208.46                 80          6   

    Flow Duration   Flow Bytes/s   Flow Packets/s   Flow IAT Mean  \
0             435            0.0      4597.701149           435.0   
1             259            0.0      7722.007722           259.0   
2             891            0.0      2244.668911           891.0   
3            1074            0.0      1862.197393          1074.0   
4             315            0.0      6349.206349           315.0   

    Flow IAT Std  ...   Bwd IAT Min  Active Mean   Active Std   Active Max  \
0            0.0  ...             0            0  

In [42]:
print(df.isnull().sum())

Source IP            0
 Source Port         0
 Destination IP      0
 Destination Port    0
 Protocol            0
 Flow Duration       0
 Flow Bytes/s        2
 Flow Packets/s      0
 Flow IAT Mean       0
 Flow IAT Std        0
 Flow IAT Max        0
 Flow IAT Min        0
Fwd IAT Mean         0
 Fwd IAT Std         0
 Fwd IAT Max         0
 Fwd IAT Min         0
Bwd IAT Mean         0
 Bwd IAT Std         0
 Bwd IAT Max         0
 Bwd IAT Min         0
Active Mean          0
 Active Std          0
 Active Max          0
 Active Min          0
Idle Mean            0
 Idle Std            0
 Idle Max            0
 Idle Min            0
label                0
dtype: int64


In [43]:
df = df.drop(['Source IP', ' Destination IP'], axis=1)

In [44]:
for i in range(len(df['label'])):
    if df['label'][i] == 'TOR':
        df['label'][i] = 1
    else:
        df['label'][i] = 0

<ipython-input-44-8054b4562e16>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = 0
<ipython-input-44-8054b4562e16>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = 1


In [45]:
for col in df.columns:
    if df[col].isin([np.inf, -np.inf]).any():
        print(col)

 Flow Bytes/s
 Flow Packets/s


In [46]:
df = df.replace([np.inf, -np.inf], np.nan)

In [47]:
thresh = 1e15

num_cols = [' Flow Duration', ' Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min',
       'Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min',
       'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean',
       ' Idle Std', ' Idle Max', ' Idle Min'] # numeric columns

for col in num_cols:
   if df[col].abs().max() > thresh:
      df[col] = df[col].clip(upper=thresh, lower=-thresh)

In [48]:
df_numeric = df.select_dtypes(include=[np.number])
thresh = 1e15

for col in df_numeric.columns:
   if df_numeric[col].abs().max() > thresh:
     df_numeric[col] = df_numeric[col].clip(upper=thresh, lower=-thresh)

df[df_numeric.columns] = df_numeric

In [49]:
imputer.fit_transform(df)

array([[5.3913e+04, 8.0000e+01, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [5.3913e+04, 8.0000e+01, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [5.3913e+04, 8.0000e+01, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [1.1365e+04, 4.4300e+02, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.3000e+04, 6.4584e+04, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [8.0000e+01, 6.5089e+04, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [51]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Protocol'] = le.fit_transform(df['Protocol'])

KeyError: ignored

In [52]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
num_features = [' Flow Duration', ' Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min',
       'Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min',
       'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean',
       ' Idle Std', ' Idle Max', ' Idle Min']
df[num_features] = scaler.fit_transform(df[num_features])

Index(['Source IP', ' Source Port', ' Destination IP', ' Destination Port',
       ' Protocol', ' Flow Duration', ' Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min',
       'Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min',
       'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean',
       ' Idle Std', ' Idle Max', ' Idle Min', 'label'],
      dtype='object')


## 1. Quantum nodes

In PennyLane, a *quantum node* is a computational unit that involves the construction, evaluation, pre- and postprocessing of quantum computations.

A quantum node consists of a *quantum function* that defines a circuit, as well as a *device* on which it is run.

There is a growing [device ecosystem](https://pennylane.ai/plugins.html) which allows you to change only one line of code to dispatch your quantum computation to local simulators, remote simulators and remote hardware from different vendors.

Here we will use the built-in `default.qubit` device.

In [ ]:
dev = qml.device('default.qubit', wires=2)

To combine the device with a quantum function to a quantum node we can use the `qml.qnode` decorator. The function can then be evaluated as if it was any other python function. Internally, it will construct a circuit and run it on the device.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    return qml.probs(wires=[0, 1])

circuit()

tensor([0.5, 0. , 0.5, 0. ], requires_grad=True)

## 2. Building quantum circuits

### The initial state

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/blob/main/figures/1.png?raw=1" width="500" height="100">
<br />

The initial state has 100% probability to be measured in the "0..0" configuration. Let's see how we can verify this with PennyLane.

In [ ]:
@qml.qnode(dev)
def circuit():
    return qml.probs(wires=[0, 1])

circuit()

tensor([1., 0., 0., 0.], requires_grad=True)

The internal state vector that we use to mathematically keep track of probabilities is complex-valued. Since `default.qubit` is a simulator we can have a look at the state, for example by checking the device's `state` attribute.

In [ ]:
dev.state

tensor([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], requires_grad=True)

### Unitary evolutions

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/blob/main/figures/2.png?raw=1" width="500">
<br />

Quantum circuits are represented by unitary matrices. We can evolve the initial state by an arbitrary unitrary matrix as follows:

In [ ]:
s = 1/np.sqrt(2)
U = np.array([[0., -s, 0.,  s],
              [ s, 0., -s, 0.],
              [ s, 0.,  s, 0.],
              [0., -s, 0., -s]])

@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.probs(wires=[0, 1])

circuit()

tensor([0. , 0.5, 0.5, 0. ], requires_grad=True)

The internal quantum state changed.

In [ ]:
dev.state

tensor([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j], requires_grad=True)

### Measurements sample outcomes from the distribution

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/blob/main/figures/3.png?raw=1" width="500">
<br />

The most common measurement takes samples $-1, 1$ from the "Pauli-Z" observable. The samples indicate if the qubit was measured in state $| 0 \rangle$ or $| 1 \rangle$.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.sample(qml.PauliZ(wires=0)), qml.sample(qml.PauliZ(wires=1))

circuit()

tensor([[ 1,  1, -1, ..., -1, -1, -1],
        [-1, -1,  1, ...,  1,  1,  1]], requires_grad=True)

The quantum state should be still the same as above.

In [ ]:
dev.state

tensor([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j], requires_grad=True)

### Computing expectation values

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/blob/main/figures/4.png?raw=1" width="500">
<br />

When we want outputs of computations to be deterministic, we often interpret the expected measurement outcome as the result. This value is estimated by taking lots of samples and averaging over them.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

tensor([0., 0.], requires_grad=True)

Again, the quantum state should be the same as above.

In [ ]:
dev.state

tensor([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j], requires_grad=True)

### Quantum circuits are decomposed into gates

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/blob/main/figures/5.png?raw=1" width="500">
<br />

Quantum circuits rarely consist of one large unitary (which quickly becomes intractably large as the number of qubits grow). Instead, they are composed of *quantum gates*.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.PauliX(wires=0)
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

tensor([ 0., -1.], requires_grad=True)

### Some gates depend on "control" parameters

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/blob/main/figures/6.png?raw=1" width="500">
<br />

To train circuits, there is a special subset of gates which is of particular interest: the Pauli rotation gates. These "rotate" a special representation of the quantum state around a specific axis. The gates depend on a scalar parameter which is the angle of the rotation.

In [ ]:
@qml.qnode(dev)
def circuit(w1, w2):
    qml.RX(w1, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(w2, wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit(1.2, 1.3)

tensor([0.36235775, 0.09693027], requires_grad=True)

The names `w1`, `w2` are already suggestive that these can be used like the trainable parameters of a classical machine learning model. But we could also call the control parameters `x1`, `x2` and encode data features into quantum states.

## 3. A full quantum machine learning model and its gradient

Finally, we can use pre-coded routines or [templates](https://pennylane.readthedocs.io/en/stable/introduction/templates.html) to conveniently build full quantum machine learning model that include a data encoding part, and a trainable part.

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/blob/main/figures/7.png?raw=1" width="500">
<br />

Here, we will use the `AngleEmbedding` template to load the data, and the `BasicEntanglingLayers` as the trainable part of the circuit.

In [ ]:
@qml.qnode(dev)
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1])
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0))


x = np.array([0.1, 0.2], requires_grad=False)
w = np.array([[-2.1, 1.2], [-1.4, -3.9], [0.5, 0.2]])

quantum_model(x, w)

tensor(0.012023, requires_grad=True)

We can draw the circuit.

In [ ]:
print(quantum_model.draw())

 0: ──RX(0.1)──RX(-2.1)──╭C──RX(-1.4)──╭C──RX(0.5)──╭C──┤ ⟨Z⟩ 
 1: ──RX(0.2)──RX(1.2)───╰X──RX(-3.9)──╰X──RX(0.2)──╰X──┤     



The best thing is that by using PennyLane, we can easily compute its gradient!

In [ ]:
gradient_fn = qml.grad(quantum_model)

gradient_fn(x, w)

(array([[ 1.69541373e-01, -2.77555756e-17],
        [ 6.97079563e-02,  2.94902991e-17],
        [ 1.69541373e-01,  1.90819582e-17]]),)

This allows us to slot the quantum circuit into the machine learning example from the previous notebook.

#  TASKS

1. Copy and paste the code from the previous notebook to here and replace the classical model by
   the `quantum_model` function. This will allow you to train the model!

2. Add a bias term to the quantum model.

A potential solution of 1 & 2 together:

In [ ]:
n_samples = 100
X0 = np.array([[np.random.normal(loc=-1, scale=1),
                np.random.normal(loc=1, scale=1)] for i in range(n_samples//2)])
X1 = np.array([[np.random.normal(loc=1, scale=1),
                np.random.normal(loc=-1, scale=1)] for i in range(n_samples//2)])

X = np.concatenate([X0, X1], axis=0)
Y = np.concatenate([-np.ones(50), np.ones(50)], axis=0)
data = list(zip(X, Y))

def model_with_bias(x, w, b):
    return quantum_model(x, w) + b

def loss(a, b):
    return (a - b)**2

def average_loss(w, b, data):
    c = 0
    for x, y in data:
        prediction = model_with_bias(x, w, b)
        c += loss(prediction, y)
    return c/len(data)

gradient_fn_w = qml.grad(average_loss, argnum=0)
gradient_fn_b = qml.grad(average_loss, argnum=1)

w_init = np.random.random(size=(3, 2))
w = np.array(w_init)
b = np.array(0.0) # start with zero bias

for i in range(15):
    w_new = w - 0.05*gradient_fn_w(w, b, data)
    b_new = b - 0.05*gradient_fn_b(w, b, data)
    print(average_loss(w_new, b_new, data))
    w = w_new
    b = b_new

1.9737277294002484
1.931550858188764
1.885820588402454
1.8367568426717318
1.7847158660372668
1.7301898352722613
1.67379472689706
1.6162456985451192
1.5583211676821624
1.5008188308188122
1.4445085381194334
1.3900877334042192
1.3381448153904907
1.2891343387585172
1.2433658449766325


3. Replace the hand-coded optimisation step by a native [PennyLane optimiser](https://pennylane.readthedocs.io/en/stable/introduction/optimizers.html).

In [ ]:
opt = qml.GradientDescentOptimizer(0.05)
for i in range(15):

    ([w, b], cst) = opt.step_and_cost(lambda w_, b_: average_loss(w_, b_, data), w, b)
    print(cst)

1.2433658449766325
1.2010058624462203
1.1620907816557215
1.1265472292920382
1.094216303496664
1.0648784356711003
1.0382764425943096
1.014135252228783
0.9921776223831842
0.972135811805237
0.9537595788243771
0.9368210978596813
0.9211174485747256
0.9064712996215246
0.8927303240804543


4. Rewrite the entire example in PyTorch.

   Tipp: You must set the qnode to the correct interface via `@qml.qnode(dev, interface='tf')`.

In [ ]:
import torch

data = [[torch.tensor(x), torch.tensor(y)] for x, y in data]


@qml.qnode(dev, interface='torch')
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1])
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0))

def average_loss(w, data):
    c = 0
    for x, y in data:
        prediction = quantum_model(x, w)
        c += loss(prediction, y)
    return c/len(data)

w_init = np.random.random(size=(3, 2))

w = torch.tensor(w_init, requires_grad=True)

opt = torch.optim.Adam([w], lr = 0.1)

# One way of optimising is to use closures
def closure():
    opt.zero_grad()
    loss = average_loss(w, data)
    loss.backward()
    return loss

for i in range(15):
    opt.step(closure)

    print(average_loss(w, data))


/home/maria/Desktop/XANADU/PennyLane demo/cern_tutorial/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


tensor(1.6454, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(1.4511, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(1.2631, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(1.0959, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.9616, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.8668, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.8105, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7836, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7733, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7689, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7646, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7593, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7539, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7500, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7483, dtype=torch.float64, grad_fn=<DivBackward0>)
